In [23]:
import cv2
import numpy as np
from scipy import ndimage

### SIFT MATCHING

In [24]:
def gaussian_kernel(k_size, sigma):
    center = k_size // 2
    x, y = np.mgrid[0 - center : k_size - center, 0 - center : k_size - center]
    g = 1 / (2 * np.pi * sigma) * np.exp(-(np.square(x) + np.square(y)) / (2 * np.square(sigma)))
    return g

In [ ]:
def non_maximum_suppression(R):
    mask = np.zeros(R.shape)
    h = R.shape[0]
    w = R.shape[1]
    d_list = [[-1, -1], [-1, 0], [-1, +1],
              [0, -1], [0, +1],
              [+1, -1], [+1, 0], [+1, +1]]

    for i in range(0,int(R.shape[0])):
        for j in range(0,int(R.shape[1])):
            flag = True
            for d in d_list:
                ni, nj = (i + d[0], j + d[1])
                if (0 <= ni < h) and (0 <= nj < w):
                    if (R[ni, nj] >= R[i, j]):
                        flag = False
            if flag:
                mask[i,j] = 1
            else:
                mask[i,j] = 0

            if(R[i-1,j-1] < R[i,j] and R[i,j-1] < R[i,j] and R[i-1,j] < R[i,j] and R[i+1,j] < R[i,j]
            and R[i-1,j+1] < R[i,j] and R[i,j+1] < R[i,j] and R[i+1,j+1] < R[i,j] and R[i+1,j-1] < R[i,j]):
                mask[i,j] = 1
            else:
                mask[i,j] = 0
    return mask

In [37]:
def get_interest_points(image, feature_width):
    small_gaussian = gaussian_kernel(9,1)
    large_gaussian = gaussian_kernel(feature_width^2, 2)

    gaussian_dx = np.gradient(small_gaussian, axis=0)
    gaussian_dy = np.gradient(small_gaussian, axis=1)
    k=0.04
    img_size = image.shape

    IX = ndimage.convolve(image, gaussian_dx)
    IY = ndimage.convolve(image, gaussian_dy)
    IX2 = ndimage.convolve(IX^2, large_gaussian)
    IY2 = ndimage.convolve(IY^2, large_gaussian)
    IXY = ndimage.convolve(IX*IY, large_gaussian)

    harris = IX2*IY2 - IXY^2 - k*(IX2+IY2)*(IX2+IY2)
    border = np.zeros(img_size)
    border[feature_width + 1 : -1 - feature_width, feature_width + 1 : -1 - feature_width ] = 1
    harris = harris * border
    threshold =  np.mean(harris)
    thresholded = harris > threshold

    mask = non_maximum_suppression (thresholded)
    
    count = np.count_nonzero(mask)
    # x = np.zeros(count,1)
    # y = np.zeros(count,1)

    x,y =np.where(mask)
    
    sum = 0
    for ii in range(1, len(x)):
        for jj in range (2,len(x)):
            sum = sum + ((x(ii)-x(jj))^2 + (y(ii)-y(jj))^2)^(1/2)
    scale = (len(x)^2)/sum
    return mask, x, y, scale

In [38]:
def get_features (image, x, y, feature_width, scale):
    feature_width = round(feature_width  * scale / 4) * 4
    total_num = len(x)
    feature_dimensionality = 128
    features = np.zeros(total_num, feature_dimensionality)
    gaussian = gaussian_kernel(feature_width,1)
    smooth_gaussian = gaussian_kernel(feature_width, feature_width/2)
    
    gaussian_dx = np.gradient(gaussian, axis=0)
    gaussian_dy = np.gradient(gaussian, axis=1)

    IX = ndimage.convolve(image, gaussian_dx)
    IY = ndimage.convolve(image, gaussian_dy)
    magnitude = np.sqrt(IX^ 2 + IY^ 2)
    direction = np.mod(round((np.arctan2(IY, IX) + 2 * np.pi) / (np.pi / 4)), 8)